## Time Series Analysis and Forecasting
### Assignment 03

Consider the Copper datasets in StatsModels.<br>
Train an appropriate model to forecast the consumption based on price, income index and aluminium price(alternative to copper)

In [29]:
copper_df = sm.datasets.copper.load_pandas()

In [30]:
copper_df.data.head(5)

,WORLDCONSUMPTION,COPPERPRICE,INCOMEINDEX,ALUMPRICE,INVENTORYINDEX,TIME
0,3173.0,26.56,0.70,19.76,0.98,1.0
1,3281.1,27.31,0.71,20.78,1.04,2.0
2,3135.7,32.95,0.72,22.55,1.05,3.0
3,3359.1,33.90,0.70,23.06,0.97,4.0
4,3755.1,42.70,0.74,24.93,1.02,5.0
